In [1]:
import time
import os
import glob
import re
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product

from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
incerto_dir = '/content/drive/MyDrive/incerto-autore'
figures_dir = os.path.join(incerto_dir, 'figures')
output_dir = os.path.join(incerto_dir, 'output')
performance_dir = os.path.join(incerto_dir, 'output','classification-performance')

## Experiments

In [4]:
def do_classification(_ids, _golden_labels, _classifier_name):

  naive_classifiers = {'Majority': DummyClassifier(strategy="most_frequent"),
                 'Minority': DummyClassifier(strategy='constant', constant=1)}

  classifier = naive_classifiers[_classifier_name]

  baseline = cross_val_score(classifier, _ids, _golden_labels, cv=3, scoring='f1_macro')

  return baseline.mean()

In [5]:
results = []
for typ in ['Split', 'Whole']:

    poems_path = os.path.join(incerto_dir, 'data', 'poems', f'poems_{typ.lower()}.csv')
    all_poems_df = pd.read_csv(poems_path)
    print(f'All {typ} poems:\t', len(all_poems_df))
    poems_df = all_poems_df.loc[all_poems_df['author'] != 'Unknown']
    print(f'Known {typ} poems:\t', len(poems_df))

    for author in poems_df['author'].unique():

        golden_labels = poems_df['author'].map(lambda x: 1 if x==author else 0).tolist()

        for classifier_name in ['Majority']: #, 'Minority']:

          score = do_classification(poems_df['label'], golden_labels, classifier_name)

          d = {'Author': author,
                'Poem format': typ,
                'Naive Classifier Type': classifier_name,
                'F-1 score': score}

          results.append(d)

All Split poems:	 1176
Known Split poems:	 1176
All Whole poems:	 622
Known Whole poems:	 622


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


In [6]:
df = pd.DataFrame(results)

In [7]:
df.groupby(['Author', 'Poem format']).max().reset_index()

,Author,Poem format,Naive Classifier Type,F-1 score
0,AntonGiacomoCorso,Split,Majority,0.487358
1,AntonGiacomoCorso,Whole,Majority,0.488066
2,BartolomeoZacco,Split,Majority,0.498935
3,BartolomeoZacco,Whole,Majority,0.497982
4,CelioMagno,Split,Majority,0.487582
5,CelioMagno,Whole,Majority,0.485099
6,DomenicoVenier,Split,Majority,0.481253
7,DomenicoVenier,Whole,Majority,0.472882
8,GiorgioGradenigo,Split,Majority,0.497221
9,GiorgioGradenigo,Whole,Majority,0.494720


In [8]:
df.groupby(['Author', 'Poem format']).max().to_csv(os.path.join(performance_dir, 'binary_baseline.csv'))